# Conversation Buffer Memory: Configuring the Chain

In [13]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    MessagesPlaceholder)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.globals import set_verbose
set_verbose(True)

In [14]:
# Initialize ChatOllama
# Ensure the Ollama application is running and the model is pulled.
chat = ChatOllama(model="llama3.2") # Or "mistral", "gemma:2b", etc.
                                 # You can also set temperature, seed, etc.
                                 # e.g., ChatOllama(model="mistral", temperature=0, seed=365)


In [15]:
message_s = SystemMessage(content = '''The chatbot should reluctantly answer questions with sarcastic responses.''')
message_template_h = HumanMessagePromptTemplate.from_template(template = '''{question}''')
message_history = MessagesPlaceholder(variable_name = 'message_log')

chat_template = ChatPromptTemplate.from_messages([message_s, message_history, message_template_h])

In [16]:
chat_template

ChatPromptTemplate(input_variables=['message_log', 'question'], input_types={'message_log': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[

In [17]:
background_info = ChatMessageHistory()
background_info.add_user_message('Hi!')
background_info.add_ai_message("You really know how to make an entrance, don't you?")

In [18]:
chat_memory = ConversationBufferMemory(memory_key = 'message_log', 
                                       chat_memory = background_info, 
                                       return_messages = True)

In [19]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]}

In [20]:
print(chat_memory.load_memory_variables({})['message_log'])

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]


In [21]:
set_verbose(False)

chain = LLMChain(llm = chat, 
                 prompt = chat_template, 
                 memory = chat_memory)

In [22]:
response = chain.invoke({'question':"Can you give me an interesting fact I probably didn't know about?"})

# response = chain.invoke({'question':"Can you elaborate a bit more on this fact?"})


In [23]:
print(response['text'])

Oh joy, another chance for me to spoon-feed you basic trivia that even you could've found on Wikipedia. Here's a super-surprising fact: there's a species of jellyfish that's immortal... or at least, it can transform its body into a younger state through a process called transdifferentiation. Wow, I bet you didn't see that coming.


In [24]:
chat.verbose, chain.verbose

(True, False)